In [ ]:
# Moravec simple implementation by G. Lisanti (not optimized)

import os
import skimage
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import io


# Sample image of a white square
img = np.zeros((100,100))
img[25:75,25:75] += 255
# img = cv2.imread("sample.jpg", cv2.IMREAD_GRAYSCALE)


# Output cornerness image
corners = np.zeros(img.shape)


# Dimension of the patch (1 = 3x3, 3 = 7x7, 5 = 11x11, ...)
w = 3


# Slide over the image
for i in range(w+1,img.shape[0]-w-1):
  for j in range(w+1,img.shape[1]-w-1):

    # Patch around the point for which the cornerness is computed
    central_patch = img[i-w:i+w+1, j-w:j+w+1]

    final_c = np.inf

    # Consider the 8 neighbors around the current pixel
    for u in [-1,0,1]:
      for v in [-1,0,1]:
        if not(u == 0 and v==0):  # avoid position(0,0)

          running_patch = img[i+u-w:i+u+w+1, j+v-w:j+v+w+1]

          # Compute the cornerness function
          c = np.sum((central_patch - running_patch)**2)

        # Save the minimum cornerness value
        if final_c > c:
          final_c = c

    corners[i,j] = final_c


# Simple thresholding
trs = corners.mean()+2*corners.std()
print("Threshold value:", trs)
trs_mask = corners > trs
corners_after_trs = corners * trs_mask

# Non maxima suppression
nms_mask = np.zeros(img.shape)
for i in range(1,corners_after_trs.shape[0]-2):
  for j in range(1,corners_after_trs.shape[1]-2):

    window = corners_after_trs[i-1:i+2,j-1:j+2]

    # if the element in the centre of the 3x3 window is not zero and is the local max
    # tip: position 4 in a vector is equal to the central position (1,1) in a 3x3 matrix
    if(window[1,1] != 0 and window.argmax()==4):
      nms_mask[i,j] = 1



corners_after_trs_nms = corners_after_trs * nms_mask

# Simple thresholding
trs = corners.mean()+2*corners.std()
print("Threshold value:", trs)
trs_mask = corners > trs
corners_after_trs = corners * trs_mask

# Show results
plt.figure(figsize=(20, 10))
plt.subplot(2, 2, 1)
plt.title("Original image")
plt.imshow(img, cmap="gray")
plt.subplot(2, 2, 3)
plt.imshow(corners)
plt.colorbar(shrink=0.7)
plt.title("Cornerness response")
plt.subplot(2, 2, 2)
plt.imshow(corners_after_trs)
plt.colorbar(shrink=0.7)
plt.title("Cornerness response after thresholding")
plt.subplot(2, 2, 4)
plt.imshow(corners_after_trs_nms)
plt.colorbar(shrink=0.7)
plt.title("Cornerness response after thresholding and non maximum suppression")
plt.show()